In [6]:
import pandas as pd
import re

# sheet_name = None opens all sheets of the input file
xlsx  = pd.read_excel(r'https://github.com/KyrilFionov/Python-Portfolio/tree/main/File-Transformation/Metrics_Forecast_Input.xlsx',sheet_name=None)

# I remove 'Model' sheet since it is not required in the analysis
xlsx.pop('Model')

# I create a variable that specifies a plan scenario I'm interested in
PLAN = 'Q4G'

regions_data = []
for region, data in xlsx.items():
    data.iloc[0:2] = data.iloc[0:2].fillna(method='ffill', axis=1)
    data = data.fillna('')
    data.columns = data.iloc[0:2].apply(lambda x: ''.join([y for y in x if y]), axis=0)
    data = data.iloc[2:]
    plan_data = (data
                 .assign(Region=region
                         ,Plan=PLAN)
                 .rename(columns=lambda data: re.sub(PLAN,'',data))
                 )
    regions_data.append(plan_data)
combined = pd.concat(regions_data)

combined = combined[[
    'Year'
    ,'Week'
    ,'Region'
    ,'Plan'
    ,'KPI 1'
    ,'KPI 2'
    ,'KPI 3'
    ,'KPI 4'
    ,'Overlay 1'
    ,'Overlay 2'
]]

# Pivoting values to later load data in the online DataBase.
# Pivoting data allows me adding more KPIs to the online table in future without having to change the structure of the table
pivot_data = pd.melt(combined,
                id_vars=['Year','Week','Region','Plan'],
                value_vars=['KPI 1','KPI 2','KPI 3','KPI 4','Overlay 1','Overlay 2'],
                value_name='value',
                var_name='metric'
                )
pivot_data.to_excel(r'https://github.com/KyrilFionov/Python-Portfolio/tree/main/File-Transformation/Metrics_Forecast_Output.xlsx', sheet_name='plan_data', index=False)